In [2]:
from time import time
import polars as pl
import pandas as pd
import numpy as np
import pyarrow

import matplotlib.pyplot as plt
import seaborn as sns

df = pl.scan_csv(r"../datasets/2019-Nov.csv")

In [23]:
%%time

# Counting all the element of the Lazy Dataframe

df.select(pl.count()).collect().item()

CPU times: total: 6.72 s
Wall time: 6.71 s


67501979

In [27]:
%%time

# Counting the unique numbers of products into the DataFrame

df.select([
    pl.col('product_id').n_unique()
]).collect(streaming=True)

CPU times: total: 13 s
Wall time: 7.18 s


product_id
u32
190662


In [30]:
%%time

# With groupby we can group elements by columns' values and perform some aggregation operations with lazy dataframe

df.groupby("brand").agg(
    pl.count().alias("num_items")
).collect(streaming=True).head()

CPU times: total: 3.44 s
Wall time: 5.94 s


brand,num_items
str,u32
"""pioneer""",201799
"""moony""",1183
"""kipocket""",1376
"""niio""",1136
"""master""",1864


In [5]:
%%time

# Aggregation with sorting

df.groupby("brand").agg([
    pl.count().alias("num_items_per_brand")
]).sort("num_items_per_brand", descending=True).collect(streaming=True).head()

CPU times: total: 6.36 s
Wall time: 6.85 s


brand,num_items_per_brand
str,u32
null,9218235
"""samsung""",7889245
"""apple""",6259379
"""xiaomi""",4638062
"""huawei""",1410126
